# Toronto Neighborhood Data

In [1]:
from bs4 import BeautifulSoup as bs
import lxml
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from keys import map_token, CLIENT_ID, CLIENT_SECRET #import private API credentials 
print("Imports complete")

Imports complete


## Import Canada Table using BeautifulSoup4

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source, 'lxml')
table = soup.table
# print(table.prettify())

bsdf = pd.read_html(str(table))[0]
bsdf

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


## Import Canada Table Data using Pandas

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]
df

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


## Clean Table Data

In [4]:
# Remove non assigned values
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)

# Change backslash to comma
df = df.applymap(lambda x: str(x.replace(' /',',')))
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df.shape

(103, 3)

## Derive Latitude and Longitude Coordinates

In [6]:
import geocoder
from pandas import json_normalize

print('Geocoder Loaded')

Geocoder Loaded


In [7]:
longitude = []
latitude = []

# Loop to find coordinates by map query
for index, row in df.iterrows():
    
    # Set up map query
    endpoint = 'Toronto, {}'.format(row['Postal code']) # Query dataframe to pull postal code
    endpoint = endpoint.replace(' ','%20') +'.json' # Replace blank spaces for query and add json identifer
    url = 'https://api.mapbox.com/geocoding/v5/mapbox.places/{}?access_token={}'.format(endpoint, map_token) # Build query url
    
    # Quergy Map API
    results = requests.get(url).json() # Query results
    results = results['features'] # Navigate to relevant results set
    
    # Capture Coordinates
    longitude.append(results[0]['center'][0])
    latitude.append(results[0]['center'][1])

print('Query Complete')

Query Complete


In [8]:
# print(latitude)
# print(longitude)

In [9]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.745820,-79.331890
1,M4A,North York,Victoria Village,43.721950,-79.304930
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.663800,-79.361550
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.721520,-79.439620
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663152,-79.389824
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.649700,-79.515290
99,M4Y,Downtown Toronto,Church and Wellesley,43.669665,-79.380257
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662910,-79.322720
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.617700,-79.499350


## Construct Mapping Environment

In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


### Generate Folium Map Object

In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Pull Foursquare Point of Interest Data

Establish base variables

In [13]:
# Set base Foursquare query parameters
VERSION = '20180605'
LIMIT = 100

Define function to query Foursquare for venues by postal codes

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=df['Postal code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [16]:
toronto_venues

,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.74582,-79.33189,PetSmart,43.748639,-79.333488,Pet Store
1,M3A,43.74582,-79.33189,Shoppers Drug Mart,43.745315,-79.325800,Pharmacy
2,M4A,43.72195,-79.30493,Sultan Of Samosas,43.718823,-79.304350,Indian Restaurant
3,M4A,43.72195,-79.30493,Tim Hortons,43.723399,-79.302647,Coffee Shop
4,M4A,43.72195,-79.30493,Subway,43.722800,-79.303465,Sandwich Place
...,...,...,...,...,...,...,...
2521,M8Z,43.62498,-79.51676,Tim Hortons,43.620900,-79.515340,Coffee Shop
2522,M8Z,43.62498,-79.51676,7-Eleven,43.629107,-79.517431,Convenience Store
2523,M8Z,43.62498,-79.51676,Buon Giorno Cafe,43.622801,-79.519322,Italian Restaurant
2524,M8Z,43.62498,-79.51676,Royal Canadian Legion #210,43.628855,-79.518903,Social Club


### Process Dataframe structure

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(2526, 7)


,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.74582,-79.33189,PetSmart,43.748639,-79.333488,Pet Store
1,M3A,43.74582,-79.33189,Shoppers Drug Mart,43.745315,-79.325800,Pharmacy
2,M4A,43.72195,-79.30493,Sultan Of Samosas,43.718823,-79.304350,Indian Restaurant
3,M4A,43.72195,-79.30493,Tim Hortons,43.723399,-79.302647,Coffee Shop
4,M4A,43.72195,-79.30493,Subway,43.722800,-79.303465,Sandwich Place


In [18]:
toronto_venues.groupby('Postal Code').count()

,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,5,5,5,5,5,5
M1C,2,2,2,2,2,2
M1E,4,4,4,4,4,4
M1G,6,6,6,6,6,6
M1H,6,6,6,6,6,6
...,...,...,...,...,...,...
M9N,10,10,10,10,10,10
M9P,11,11,11,11,11,11
M9R,13,13,13,13,13,13


In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 263 uniques categories.


## Analyze Neighborhoods

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal code,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waste Facility,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_onehot.shape

(2526, 264)

In [22]:
toronto_grouped = toronto_onehot.groupby('Postal code').mean().reset_index()
toronto_grouped

,Postal code,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waste Facility,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M9N,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
97,M9P,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,M9R,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,M9V,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
toronto_grouped.shape

(101, 264)

## Print top 5 venues by postal code

In [24]:
num_top_venues = 5

for code in toronto_grouped['Postal code']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['Postal code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0                  Park   0.2
1  Fast Food Restaurant   0.2
2           Bus Station   0.2
3           Gas Station   0.2
4          Intersection   0.2


----M1C----
                             venue  freq
0       Construction & Landscaping   0.5
1                              Bar   0.5
2                        Nightclub   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Monument / Landmark   0.0


----M1E----
               venue  freq
0              Hotel  0.25
1  Convenience Store  0.25
2               Park  0.25
3     Breakfast Spot  0.25
4     Ice Cream Shop  0.00


----M1G----
                    venue  freq
0          Sandwich Place  0.17
1    Fast Food Restaurant  0.17
2  Thrift / Vintage Store  0.17
3        Department Store  0.17
4             Supermarket  0.17


----M1H----
               venue  freq
0  Indian Restaurant  0.33
1  Recreation Center  0.17
2             Bakery  0.17
3  Fish & Chips Shop  0.17
4        Snack Pl

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
code_venues_sorted = pd.DataFrame(columns=columns)
code_venues_sorted['Postal code'] = toronto_grouped['Postal code']

for ind in np.arange(toronto_grouped.shape[0]):
    code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

code_venues_sorted.head()

,Postal code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Intersection,Gas Station,Fast Food Restaurant,Park,Bus Station,Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
1,M1C,Construction & Landscaping,Bar,Dive Bar,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Farm
2,M1E,Convenience Store,Hotel,Breakfast Spot,Park,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
3,M1G,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Thrift / Vintage Store,Supermarket,Department Store,Yoga Studio,Event Space,Donut Shop,Eastern European Restaurant
4,M1H,Indian Restaurant,Bakery,Fish & Chips Shop,Snack Place,Recreation Center,Event Space,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store


In [27]:
# Set cluster
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 3, 0, 0, 4, 0, 3, 0, 4], dtype=int32)

In [28]:
# add clustering labels
code_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(code_venues_sorted.set_index('Postal code'), on='Postal code')

# clean any NaN values
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.745820,-79.331890,2.0,Pharmacy,Pet Store,Yoga Studio,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market
1,M4A,North York,Victoria Village,43.721950,-79.304930,0.0,Indian Restaurant,Pet Store,Pharmacy,Rental Car Location,Beer Store,Pizza Place,Sandwich Place,Shopping Mall,Coffee Shop,Food
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.663800,-79.361550,0.0,Park,Restaurant,Pool,Farmers Market,Performing Arts Venue,Farm,Sushi Restaurant,Baseball Field,Fast Food Restaurant,Thai Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.721520,-79.439620,4.0,Park,Lawyer,Light Rail Station,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663152,-79.389824,0.0,Coffee Shop,Sushi Restaurant,Park,Bank,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Distribution Center,Beer Bar,Fried Chicken Joint


In [29]:
# create map
map_clusters = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Toronto Clusters

### Cluster 1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Indian Restaurant,Pet Store,Pharmacy,Rental Car Location,Beer Store,Pizza Place,Sandwich Place,Shopping Mall,Coffee Shop,Food
2,Downtown Toronto,0.0,Park,Restaurant,Pool,Farmers Market,Performing Arts Venue,Farm,Sushi Restaurant,Baseball Field,Fast Food Restaurant,Thai Restaurant
4,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Park,Bank,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Distribution Center,Beer Bar,Fried Chicken Joint
8,East York,0.0,Chinese Restaurant,Grocery Store,Pet Store,Burger Joint,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
9,Downtown Toronto,0.0,Coffee Shop,Gastropub,Cosmetics Shop,Café,Italian Restaurant,Theater,Burrito Place,Sandwich Place,Restaurant,Furniture / Home Store
...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,0.0,Italian Restaurant,Pizza Place,Coffee Shop,Indie Movie Theater,Thai Restaurant,Pub,Spa,Dessert Shop,Pool Hall,Rental Car Location
99,Downtown Toronto,0.0,Coffee Shop,Grocery Store,Restaurant,Café,Japanese Restaurant,Hotel,Sandwich Place,Breakfast Spot,Salad Place,Food & Drink Shop
100,East Toronto,0.0,Park,Yoga Studio,Smoke Shop,Garden Center,Beer Store,Light Rail Station,Spa,Burrito Place,Fast Food Restaurant,Bakery
101,Etobicoke,0.0,Bakery,Performing Arts Venue,Diner,Bar,Financial or Legal Service,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Dive Bar


### Cluster 2

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,1.0,Playground,Hardware Store,Bus Line,Yoga Studio,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
10,North York,1.0,Spa,Home Service,Playground,Trail,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
85,Scarborough,1.0,Sandwich Place,Trail,Yoga Studio,Event Space,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Distribution Center


### Cluster 3

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Pharmacy,Pet Store,Yoga Studio,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market


### Cluster 4

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Scarborough,3.0,Convenience Store,Hotel,Breakfast Spot,Park,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant
35,East York,3.0,Park,Cheese Shop,Bus Line,Breakfast Spot,Convenience Store,Curling Ice,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop
44,Scarborough,3.0,Convenience Store,Restaurant,Park,Ice Cream Shop,Dessert Shop,Yoga Studio,Falafel Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store
56,York,3.0,Convenience Store,Park,Fast Food Restaurant,Caribbean Restaurant,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
82,Scarborough,3.0,Convenience Store,Park,Caribbean Restaurant,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market
91,Downtown Toronto,3.0,Park,Trail,Convenience Store,Metro Station,Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant


### Cluster 5

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4.0,Park,Lawyer,Light Rail Station,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market
6,Scarborough,4.0,Intersection,Gas Station,Fast Food Restaurant,Park,Bus Station,Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space
7,North York,4.0,Café,Botanical Garden,Park,Yoga Studio,Farm,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant
14,East York,4.0,Pharmacy,Park,Skating Rink,Bus Line,Cosmetics Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Dive Bar,Farm
32,Scarborough,4.0,Grocery Store,Spa,Park,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Yoga Studio
36,Downtown Toronto,4.0,Pier,Harbor / Marina,Café,Park,Disc Golf,Creperie,Cupcake Shop,Financial or Legal Service,Filipino Restaurant,Fast Food Restaurant
41,East Toronto,4.0,Park,Café,Business Service,Grocery Store,Bar,Bakery,Tailor Shop,Discount Store,Chinese Restaurant,Fast Food Restaurant
58,Scarborough,4.0,Thai Restaurant,Diner,Park,Beach,Event Space,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant
60,North York,4.0,Food & Drink Shop,Park,Tea Room,Arts & Crafts Store,Yoga Studio,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
61,Central Toronto,4.0,Photography Studio,Park,Yoga Studio,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market
